In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
!pip3 install loralib

In [ ]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
from datasets import load_dataset
from transformers import get_scheduler
import loralib as lora
from peft import LoraConfig, AdaLoraModel, AdaLoraConfig
import time

from datasets import concatenate_datasets

from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd

# Create dataset

In [ ]:
# Load the IMDB dataset
dataset = load_dataset("imdb")

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the IMDB dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# Sample 2500 from the first 12500 negative samples
negative_samples_train = train_dataset.select(range(12500)).shuffle(seed=42).select(range(2500))

# Sample 2500 from the second 12500 positive samples
positive_samples_train = train_dataset.select(range(12500, 25000)).shuffle(seed=42).select(range(2500))

balanced_train_dataset = concatenate_datasets([negative_samples_train, positive_samples_train])

In [ ]:
# Sample 2500 from the first 12500 negative samples
negative_samples_test = test_dataset.select(range(12500)).shuffle(seed=42).select(range(2500))

# Sample 2500 from the second 12500 positive samples
positive_samples_test = test_dataset.select(range(12500, 25000)).shuffle(seed=42).select(range(2500))

balanced_test_dataset = concatenate_datasets([negative_samples_test, positive_samples_test])

In [ ]:
balanced_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})

In [ ]:
balanced_test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary", zero_division=1)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [ ]:
5e7abf784a3f7805725b2a241d1a5b31d222be1d

# Adalora

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import (
    get_peft_model,
    AdaLoraConfig,
    TaskType,
)

In [ ]:
# Define model name, number of epochs, learning rate, and batch size
model_name = "distilbert-base-uncased"
num_epochs = 3
lr = 2e-5
batch_size = 8

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define AdaLoRA Configuration
adalora_config = AdaLoraConfig(
    task_type=TaskType.SEQ_CLS,    # Task type is sequence classification
    r=8,                           # Rank parameter, can be tuned
    lora_alpha=32,                      # Scaling factor, can be tuned
    target_modules=["q_lin", "v_lin"],  # Target modules for low-rank adaptation
    lora_dropout=0.1,                   # Dropout rate for AdaLoRA
    target_r=4,                    # Target rank for adaptive procedure
    beta1=0.85,                    # Beta1 parameter for AdaLoRA
    beta2=0.85,                    # Beta2 parameter for AdaLoRA
    tinit=200,                     # The number of steps before AdaLoRA starts adapting
    tfinal=1000,                   # The number of steps after which adaptation stops
    deltaT=10,                     # Frequency of rank adjustment
)

# Apply AdaLoRA to the model
model = get_peft_model(model, adalora_config)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model.print_trainable_parameters()

trainable params: 813,458 || all params: 67,768,480 || trainable%: 1.2003


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="Adalora_distilbert_imdb",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=balanced_train_dataset,
    eval_dataset=balanced_test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall: {eval_results['eval_recall']:.4f}")
print(f"F1 Score: {eval_results['eval_f1']:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.602000,0.319201,0.908806,0.813200,0.858349
2,0.351100,0.288267,0.876659,0.898400,0.887396


In [ ]:
print("\nEvaluation Results:")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall: {eval_results['eval_recall']:.4f}")
print(f"F1 Score: {eval_results['eval_f1']:.4f}")


Evaluation Results:
Precision: 0.8970
Recall: 0.8852
F1 Score: 0.8911


In [ ]:
eval_results

{'eval_loss': 0.2746381461620331,
 'eval_precision': 0.8970409404134576,
 'eval_recall': 0.8852,
 'eval_f1': 0.8910811354942622,
 'eval_runtime': 77.189,
 'eval_samples_per_second': 64.776,
 'eval_steps_per_second': 8.097,
 'epoch': 3.0}